In [1]:
import collections
import copy
import cspUtil
import dataUtil

In [2]:
def countSequences(inputList, N):
    out = collections.defaultdict(float)
    for i in range(len(inputList)-N+1):
        if [inputList[i+k] != None for k in range(N)] == [True for _ in range(N)]:
            seq = tuple([inputList[i+k] for k in range(N)])
            out[seq] += 1
    print out

    factor= 1.0 / sum(out.itervalues())
    for k in out:
          out[k] *= 10*factor

    return out

In [3]:
midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/chor_armure.mid"
window = (10, 15)
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()
rythmUnit, windowedPrimTrack = mypattern.getCorrupt(window)

Loaded /Users/robin/Desktop/CS221_projet/Open_MIDI_file/chor_armure.mid


In [4]:
def createRythmCSP(windowedPrimTrack, window, bi=True, tri=False):
    cspRythm = cspUtil.CSP()
    (start, end) = window
    # Creating "units" variables
    transitions = [note[1] for note in windowedPrimTrack]
    for unitNum in range(start, end+1):
        unit = windowedPrimTrack[unitNum]
        cspRythm.add_variable(('U', unitNum), [(unit[0],True),(unit[0],False)])

    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(transitions, 2)
        #print "bi"
        #print biSequences
        for varNum in range(start, end):
            cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1), lambda u1, u2 : biSequences[(u1[1], u2[1])])
        # Add corner cases
        cspRythm.add_unary_factor(('U', start), lambda u : biSequences[(windowedPrimTrack[start-1][1], u[1])])
        cspRythm.add_unary_factor(('U', end)  , lambda u : biSequences[(u[1], windowedPrimTrack[end+1][1])])

    if tri:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(start, end+1):
            cspRythm.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(transitions, 3)

        for varNum in range(start, end+1):
            # Add constraint B_i[0] = U_i[1]
            cspRythm.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != end:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspRythm.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspRythm.add_unary_factor(('B', start), lambda b: b[1] == windowedPrimTrack[start-1])
        cspRythm.add_unary_factor(('B', end)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[end+1])] \
                        * triSequences[(b[0], windowedPrimTrack[end+1], windowedPrimTrack[end+2])])

    return cspRythm

In [60]:
def createPitchCSP(windowedPitchPrimTrack, window, bi=True, tri=False):
    (start, end) = window
    cspPitch = cspUtil.CSP()
    pitchTrack = [note[0] for note in windowedPrimTrack]
    # Creating "units" variables #
    # Domains
    minPitch, maxPitch = pitchTrack[0], pitchTrack[0]
    for pitch in pitchTrack:
        if type(pitch) == int:
            minPitch, maxPitch = min(minPitch, pitch), max(maxPitch, pitch)
    domains = tuple(list(range(minPitch, maxPitch+1)) + ['silence'])
    
    for pitchNum in range(start, end+1):
        cspPitch.add_variable(('U', pitchNum), copy.deepcopy(domains))

     #frequency of single apparition
    freq1 = countSequences(pitchTrack, 1)
    for pitchIdx in range(start, end):
        cspPitch.add_unary_factor(('U', pitchIdx), lambda p : freq1[(p,)])
        
    # Create binary factor: must keep the same pitch if previous duration = True
    for varNum in range(start, end): 
        if windowedPitchPrimTrack[varNum][1]:
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda x, y : x == y)
    
    
    # Create binary factors proportional to frequence of apparition of pairs
    biSequences = countSequences(pitchTrack, 2)
    #print "bi"
    #print biSequences
    for varNum in range(start, end):
        cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda p1, p2 : biSequences[(p1, p2)])
    # Add corner cases
    cspPitch.add_unary_factor(('U', start), lambda p : biSequences[(pitchTrack[start-1], p)])
    cspPitch.add_unary_factor(('U', end)  , lambda p : biSequences[(p, pitchTrack[end+1])])
    # must keep same pitch if previous duration = True
    if windowedPitchPrimTrack[start-1][1]:
        cspPitch.add_unary_factor(('U', start), lambda x : x == windowedPitchPrimTrack[start-1][0])
    if windowedPitchPrimTrack[end][1]:
        cspPitch.add_unary_factor(('U', end), lambda x : x == windowedPitchPrimTrack[end+1][0])

    """
    if triSeq:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(start, end+1):
            cspPitch.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(windowedPrimTrack, 3)

        for varNum in range(start, end+1):
            # Add constraint B_i[0] = U_i[1]
            cspPitch.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != end:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspPitch.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspPitch.add_unary_factor(('B', start), lambda b: b[1] == windowedPrimTrack[start-1])
        cspPitch.add_unary_factor(('B', end)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[end+1])] \
                        * triSequences[(b[0], windowedPrimTrack[end+1], windowedPrimTrack[end+2])])
        """

    return cspPitch

In [61]:
midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid"
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()

Loaded /Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid


In [62]:
def reconstruction(windowedPrimTrack, window, bi=True, tri=False):
    # Rhythm reconstruction
    CSP_rythm = createRythmCSP(windowedPrimTrack, window, bi, tri)
    search_rythm = cspUtil.BacktrackingSearch()
    search_rythm.solve(CSP_rythm)
    windowedPitchPrimTrack = windowedPrimTrack[:]
    for i, note in enumerate(windowedPitchPrimTrack[:]):
        if note == (None, None):
            windowedPitchPrimTrack[i] = search_rythm.optimalAssignment[('U', i)]
    
    # Pitch reconstruction
    CSP_pitch = createPitchCSP(windowedPitchPrimTrack, window, bi, tri)
    search_pitch = cspUtil.BacktrackingSearch()
    search_pitch.solve(CSP_pitch)
    primReconstruction = windowedPitchPrimTrack[:]
    for i, note in enumerate(primReconstruction[:]):
        if note[0] == None:
            primReconstruction[i] = (search_pitch.optimalAssignment[('U', i)],primReconstruction[i][1])
    
    return primReconstruction

In [63]:
def lossFunction(window, primOriginal, primReconstruction):
    (start, end) = window
    accuracy = 0
    for i in range(start, end+1):
        if primOriginal[i][0] == primReconstruction[i][0]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original pitch:', primOriginal[i][0], 'Reconstructed pitch:', primReconstruction[i][0]
            
        if primOriginal[i][1] == primReconstruction[i][1]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original rhythm:', primOriginal[i][1], 'Reconstructed rhythm:', primReconstruction[i][1]
            
    loss = 100 * (1 - accuracy/(2.*(end+1-start)))
    print 'Loss value (percentage):', loss
    return  loss

In [66]:
trackNum = 0
window = (20, 30)
rythmUnit, windowedPrimTrack = mypattern.getCorrupt(window, trackNum=0)
primReconstruction = reconstruction(windowedPrimTrack, window, bi=True, tri=False)
primOriginal = mypattern.primTracks[trackNum]
lossFunction(window, primOriginal, primReconstruction)

defaultdict(<type 'float'>, {(False, True): 23.0, (True, False): 24.0, (False, False): 1.0, (True, True): 3.0})
Found 1 optimal assignments with weight 91367628.606887 in 4095 operations
First assignment took 12 operations
One of the optimal assignments is
('U', 20) (None, True)
('U', 21) (None, False)
('U', 22) (None, True)
('U', 23) (None, False)
('U', 24) (None, True)
('U', 25) (None, False)
('U', 26) (None, True)
('U', 27) (None, False)
('U', 28) (None, True)
('U', 29) (None, False)
('U', 30) (None, True)
defaultdict(<type 'float'>, {(79,): 8.0, (74,): 10.0, (77,): 8.0, (72,): 13.0, (76,): 14.0})
defaultdict(<type 'float'>, {(74, 72): 3.0, (77, 76): 2.0, (76, 76): 9.0, (77, 77): 4.0, (74, 74): 5.0, (76, 77): 2.0, (77, 79): 2.0, (74, 76): 2.0, (72, 74): 1.0, (79, 77): 2.0, (76, 74): 3.0, (72, 72): 10.0, (79, 79): 6.0})
Found 1 optimal assignments with weight 1272924.713323 in 377 operations
First assignment took 12 operations
One of the optimal assignments is
('U', 20) 72
('U', 21) 

59.09090909090908